# Notebook for datasets generation

In [1]:
import os
outdir = os.path.join(os.environ["HOME"], "notebook_dataset_generation_for_models_output")
csvdir = os.path.join(outdir, "csv")
pngdir = os.path.join(outdir, "png")
! mkdir -p $csvdir                          
! mkdir -p $pngdir                          
! ls $outdir

import collections
import numpy as np
import pandas as pd
from os import listdir
from pathlib import Path
from astropy.io import fits
from os.path import isfile, join

import matplotlib.pyplot as plt
%matplotlib widget

datapath = Path("/data01/home/baroncelli/phd/DATA")
os.environ["DATA"] = str(datapath)
from RTAscience.cfg.Config import Config

from RTAscience.lib.RTAUtils import get_pointing
from rtapipe.lib.datasource.photometry import Photometry
from rtapipe.lib.plotting.photometryplot import PhotometrySinglePlot, PhotometrySubPlots

csv  png



Bad key "text.kerning_factor" on line 4 in
/data01/home/baroncelli/.conda/envs/bphd/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.2/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
def getInput(dataDir, howMany=-1):

    simFolder = datapath.joinpath("obs", dataDir)
    cfg = Config(simFolder.joinpath("config.yaml"))
    runid = cfg.get('runid')
    template =  os.path.join(datapath, f'templates/{runid}.fits')
    pointing = get_pointing(template)
    
    
    #if cfg.get("simtype") == 'bkg':
    #    dir = simFolder.joinpath("backgrounds")
    #else:
    #    dir = simFolder.joinpath(runid)
                                                        
    datafiles = [join(simFolder, f) for f in listdir(simFolder) 
                    if isfile(join(simFolder, f)) and ".yaml" not in f and ".log" and "IRF" not in f] 
    
    sim_params = []
    for datafile in datafiles:
        
        conf = {
            'input_file': datafile,
            'output_dir': outdir,
            'simtype' : cfg.get('simtype'),
            'runid' : cfg.get('runid'),
            't_window_start': 0,
            't_window_stop': cfg.get('tobs'),
            'e_window_start': cfg.get('emin'),
            'e_window_stop': cfg.get('emax'),
            'onset' : cfg.get('onset')
        }
        sim_params.append(conf)

        if len(sim_params) >= howMany:
            break
            
            
    print(f"Found: {len(sim_params)} files, pointing is: {pointing}")

    return sim_params, pointing


## Background only 

In [3]:
photometry = Photometry()

photometry_params = {
    
    # integration parameters for time
    't_window_size': 10,
    't_window_step': 10,
    
    # integration parameters for energy    
    'e_window_size': 0.001,
    'e_window_step': 0.001,    

    # Parameters that can change too    
    'pointing' : None,
    'region_radius': 1.0,
    
    # Other settings
    'plot' : 1,
    'override': 1
}

In [4]:
bkgonly = "autoencoder/run0406_ID000126_OK_bkg_only"
sim_params, pointing = getInput(bkgonly, 100)
print("pointing: ",pointing)
photometry_params['pointing'] = pointing

Found: 100 files, pointing is: (33.057, -51.841)
pointing:  (33.057, -51.841)


In [5]:
print("photometry_params")
_ = [print(f"\t{k}: {v}") for k,v in photometry_params.items()]
print("sim_params[0]")
_ = [print(f"\t{k}: {v}") for k,v in sim_params[0].items()]

photometry_params
	t_window_size: 10
	t_window_step: 10
	e_window_size: 0.001
	e_window_step: 0.001
	pointing: (33.057, -51.841)
	region_radius: 1.0
	plot: 1
	override: 1
sim_params[0]
	input_file: /data01/home/baroncelli/phd/DATA/obs/autoencoder/run0406_ID000126_OK_bkg_only/bkg000001.log
	output_dir: /data01/home/baroncelli/notebook_dataset_generation_for_models_output
	simtype: bkg
	runid: run0406_ID000126
	t_window_start: 0
	t_window_stop: 1800
	e_window_start: 0.03
	e_window_stop: 0.15
	onset: 0


In [6]:
singleplot = PhotometrySinglePlot(title="Multiple trials")
for i in range(1,5):
    csv_data_t = photometry.integrate(photometry_params, sim_params[i], integration="t").pop(0)  
    _ = singleplot.addData(csv_data_t, photometry_params, sim_params[i], [f"trial {i}"], "t")


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

input_file:  /data01/home/baroncelli/phd/DATA/obs/autoencoder/run0406_ID000126_OK_bkg_only/bkg000002.log


OSError: Header missing END card.

## GRB after onset

In [ ]:
photometry = Photometry()

photometry_params = {
    
    # integration parameters for time
    't_window_size': 10,
    't_window_step': 10,
    
    # integration parameters for energy    
    'e_window_size': 0.001,
    'e_window_step': 0.001,    

    # Parameters that can change too    
    'pointing' : None,
    'region_radius': 1.0,
    
    # Other settings
    'plot' : 1,
    'override': 1
}

In [ ]:
datadir = "autoencoder/run0406_ID000126_OK_bkg_src"
sim_params, pointing = getInput(datadir, 100)

photometry_params['pointing'] = pointing
print(sim_params[0])

In [ ]:
print("photometry_params")
_ = [print(f"\t{k}: {v}") for k,v in photometry_params.items()]
print("sim_params[0]")
_ = [print(f"\t{k}: {v}") for k,v in sim_params[0].items()]

In [ ]:
singleplot = PhotometrySinglePlot(title="Multiple trials")
for i in range(1,5):
    csv_data_t = photometry.integrate(photometry_params, sim_params[i], integration="t").pop(0)  
    _ = singleplot.addData(csv_data_t, photometry_params, sim_params[i], [f"trial {i}"], "t")